# 01 — Gold Loader + QC (USD/CAD)

**Goal:** Load the Gold-layer parquet for USD/CAD, validate its integrity, and build a **model-ready dataset** for directional forecasting.

This notebook produces:
- a clean `df` indexed by `obs_date`
- a 7-business-day directional target (`target_direction_7d`)
- a feature matrix `X` and labels `y` ready for rolling backtests


## 1) Imports

We use:
- `pyarrow` to read parquet reliably
- `pandas/numpy` for cleaning and transformations

> Note: Notebooks are order-sensitive. Always run top-to-bottom after changes.


In [2]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from pathlib import Path


## 2) Paths (Repo-safe)

We avoid absolute paths so the notebook can run on any machine without leaking local directories into Git.

We locate the repo root by searching upward for folders like `data/` and `src/`, then define file paths relative to that root.


In [12]:
from pathlib import Path

def find_repo_root(start: Path | None = None) -> Path:
    start = start or Path.cwd()
    for p in [start, *start.parents]:
        if (p / "data").exists() and (p / "src").exists():
            return p
    raise RuntimeError("Repo root not found. Run the notebook from inside the repo.")

REPO_ROOT = find_repo_root()
DATA_DIR = REPO_ROOT / "data"

PARQUET_PATH = DATA_DIR / "data-USD-CAD.parquet"

print("Repo root:", REPO_ROOT)
print("Parquet:", PARQUET_PATH)

def load_gold_parquet(path: Path, series_id: str = "FXUSDCAD") -> pd.DataFrame:
    """
    Load Gold-layer parquet and return a clean, date-indexed DataFrame for one series_id.
    """
    # Read parquet safely using pyarrow (avoids pandas extension quirks)
    table = pq.read_table(str(path))
    df = table.to_pandas()

    # Basic filtering
    if "series_id" in df.columns:
        df = df[df["series_id"] == series_id].copy()

    # Parse date
    if "obs_date" not in df.columns:
        raise ValueError("Expected column 'obs_date' in gold parquet.")

    df["obs_date"] = pd.to_datetime(df["obs_date"])
    df = df.sort_values("obs_date").reset_index(drop=True)
    df = df.set_index("obs_date")

    # Ensure numeric columns are numeric
    for c in df.columns:
        if c in ("series_id", "base_currency", "quote_currency", "source", "run_id", "processed_at"):
            continue
        try:
            df[c] = pd.to_numeric(df[c])
        except Exception:
            pass


    # Minimal sanity checks
    if "value" not in df.columns:
        raise ValueError("Expected column 'value' (FX rate) in gold parquet.")
    if df["value"].isna().mean() > 0.01:
        print("Warning: 'value' contains >1% NaNs. Consider investigating upstream.")
    df = df[~df["value"].isna()].copy()

    return df

df = load_gold_parquet(PARQUET_PATH, series_id="FXUSDCAD")
df.head()


Repo root: /Users/ianvicente/Desktop/FX-Rate-Forecasting-Pipeline
Parquet: /Users/ianvicente/Desktop/FX-Rate-Forecasting-Pipeline/data/data-USD-CAD.parquet


,series_id,base_currency,quote_currency,value,prev_value,daily_return,log_return,return_5d,return_21d,lag_1d,...,is_month_end,is_quarter_end,is_year_start,is_year_end,target_return_1d,target_direction_1d,target_return_5d,source,run_id,processed_at
obs_date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,FXUSDCAD,USD,CAD,1.3435,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,False,-0.008972,0,-0.016524,bankofcanada_valet,gold_backfill_20251223T210750Z,2025-12-23T21:12:37.627684+00:00
2017-01-04,FXUSDCAD,USD,CAD,1.3315,1.3435,-0.008932,-0.008972,NaN,NaN,1.3435,...,False,False,False,False,-0.005347,0,-0.004882,bankofcanada_valet,gold_backfill_20251223T210750Z,2025-12-23T21:12:37.627684+00:00
2017-01-05,FXUSDCAD,USD,CAD,1.3244,1.3315,-0.005332,-0.005347,NaN,NaN,1.3315,...,False,False,False,False,-0.002268,0,-0.010420,bankofcanada_valet,gold_backfill_20251223T210750Z,2025-12-23T21:12:37.627684+00:00
2017-01-06,FXUSDCAD,USD,CAD,1.3214,1.3244,-0.002265,-0.002268,NaN,NaN,1.3244,...,False,False,False,False,0.001966,1,-0.005524,bankofcanada_valet,gold_backfill_20251223T210750Z,2025-12-23T21:12:37.627684+00:00
2017-01-09,FXUSDCAD,USD,CAD,1.3240,1.3214,0.001968,0.001966,NaN,NaN,1.3214,...,False,False,False,False,-0.002041,0,-0.006647,bankofcanada_valet,gold_backfill_20251223T210750Z,2025-12-23T21:12:37.627684+00:00


## 3) Load Gold Parquet

We load the Gold-layer parquet and filter to the `FXUSDCAD` series.

We enforce:
- datetime index (`obs_date`)
- sorted chronology
- numeric conversion where appropriate
- removal of invalid `value` rows


## 4) Data QC (Quick sanity checks)

We run fast checks to confirm the dataset is suitable for modeling:
- row count and date range
- missingness and basic descriptive stats
- expected business-day behavior (no weekends)

This is intentionally lightweight: just enough to catch obvious issues before modeling.


In [8]:
print("Rows:", len(df))
print("Date range:", df.index.min().date(), "→", df.index.max().date())
print("Columns:", len(df.columns))

# Check business-day gaps (expected)
gaps = df.index.to_series().diff().value_counts().head(10)
print("\nTop index gaps:\n", gaps)

# Missingness overview
missing = df.isna().mean().sort_values(ascending=False)
print("\nTop missing columns:\n", missing.head(15))


Rows: 2239
Date range: 2017-01-03 → 2025-12-22
Columns: 38

Top index gaps:
 obs_date
1 days    1756
3 days     392
4 days      74
2 days      11
5 days       5
Name: count, dtype: int64

Top missing columns:
 lag_21d             0.009379
return_21d          0.009379
lag_5d              0.002233
return_5d           0.002233
target_return_5d    0.002233
rolling_std_21d     0.001787
volatility_ratio    0.001787
lag_3d              0.001340
lag_2d              0.000893
log_return          0.000447
daily_return        0.000447
target_return_1d    0.000447
prev_value          0.000447
rolling_std_5d      0.000447
lag_1d              0.000447
dtype: float64


## 5) Define the modeling target (H = 7 business days)

Instead of forecasting the exact USD/CAD level (hard in FX), we frame the task as:

- **Return over 7 business days**
- **Direction label**: up vs down/flat

This sets us up for a “direction + confidence” approach in later notebooks.


In [14]:
H = 7  # horizon in business days

# forward return over H steps
df["target_return_7d"] = df["value"].shift(-H) / df["value"] - 1.0

# direction label: 1 if up, 0 if down/flat
df["target_direction_7d"] = (df["target_return_7d"] > 0).astype(int)

# Drop last H rows where target is NaN
df_model = df.iloc[:-H].copy()

df_model[["value", "target_return_7d", "target_direction_7d"]].tail(10)


,value,target_return_7d,target_direction_7d
obs_date,,,
2025-11-28,1.3979,-0.009657,0
2025-12-01,1.3979,-0.010301,0
2025-12-02,1.3986,-0.015158,0
2025-12-03,1.3949,-0.012904,0
2025-12-04,1.3952,-0.013045,0
2025-12-05,1.3860,-0.008081,0
2025-12-08,1.3837,-0.003975,0
2025-12-09,1.3844,-0.005056,0
2025-12-10,1.3835,-0.003903,0


## 6) Feature set (disciplined)

We start with a conservative feature set:
- returns and log returns
- lags
- rolling means / rolling volatility
- calendar features (day-of-week, month-end, etc.)

We intentionally keep features limited early to:
- reduce leakage risk
- keep baselines interpretable
- make backtests fast and stable


In [17]:
FEATURES = [
    # Returns / lags
    "daily_return", "log_return", "return_5d", "return_21d",
    "lag_1d", "lag_2d", "lag_3d", "lag_5d", "lag_21d",

    # Rolling stats
    "rolling_mean_5d", "rolling_mean_21d",
    "rolling_std_5d", "rolling_std_21d",
    "volatility_ratio",

    # Simple derived indicator (if present)
    "ma_crossover",

    # Calendar
    "day_of_week", "day_of_month", "week_of_year", "month", "quarter",
    "is_month_start", "is_month_end", "is_quarter_end",
    "is_year_start", "is_year_end",
]

# Keep only features that actually exist in the parquet
FEATURES = [c for c in FEATURES if c in df_model.columns]
print("Using features:", FEATURES)
print("Feature count:", len(FEATURES))


Using features: ['daily_return', 'log_return', 'return_5d', 'return_21d', 'lag_1d', 'lag_2d', 'lag_3d', 'lag_5d', 'lag_21d', 'rolling_mean_5d', 'rolling_mean_21d', 'rolling_std_5d', 'rolling_std_21d', 'volatility_ratio', 'ma_crossover', 'day_of_week', 'day_of_month', 'week_of_year', 'month', 'quarter', 'is_month_start', 'is_month_end', 'is_quarter_end', 'is_year_start', 'is_year_end']
Feature count: 25


## 7) Build model-ready matrices

We create:
- `X`: feature matrix
- `y`: direction labels (`target_direction_7d`)

We handle:
- `inf` values (replace with NaN)
- missing values (forward-fill, then fallback to 0)

Finally we check:
- dataset shape
- class balance (positive class rate)


In [19]:
X = df_model[FEATURES].copy()
y = df_model["target_direction_7d"].copy()

# Basic cleaning: fill remaining NaNs (from rolling features early in series)
X = X.replace([np.inf, -np.inf], np.nan)

# Simple, safe imputation for now (we can refine later)
X = X.ffill().fillna(0.0)

print("X shape:", X.shape, "y shape:", y.shape)
print("Positive class rate:", y.mean())


X shape: (2232, 25) y shape: (2232,)
Positive class rate: 0.5085125448028673
